<a href="https://colab.research.google.com/github/AbdulxoliqMirzayev/STT_NER_MODEL_PROJECT/blob/main/Copy_of_STT_NER_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch datasets transformers librosa jiwer soundfile openai-whisper evaluate bitsandbytes accelerate gradio

# Kutubxonalarni import qilish
import torch
import os
import re
import numpy as np
import librosa
import soundfile as sf
import IPython.display as ipd
import evaluate
from scipy.signal import butter, lfilter
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import load_dataset, Dataset, DatasetDict, Audio
from transformers import (
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperFeatureExtractor,
    AutomaticSpeechRecognitionPipeline,
    BitsAndBytesConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from huggingface_hub import HfApi
import gradio as gr
from peft import (
    PeftModel,
    PeftConfig,
    prepare_model_for_kbit_training,
    LoraConfig,
    LoraModel,
    get_peft_model
)

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small", language="uz", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="uz", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="uz", task="transcribe")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.7 MB/s et

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [2]:
# Common Voice datasetini yuklash
common_voice = "mozilla-foundation/common_voice_11_0"



**Ushbu kod Common Voice’dan o‘zbek tilidagi nutq ma’lumotlarini yuklash uchun ishlatiladi. Ma’lumotlar train+validation va test qismlarga ajratiladi va DatasetDict obyektida saqlanadi. Bu strukturali ma’lumotlar keyinchalik STT modeli uchun ishlatiladi.**

In [3]:
data_dict = DatasetDict()

data_dict['train'] = load_dataset(common_voice,"uz", split="train+validation")
data_dict['test'] = load_dataset(common_voice,"uz", split="test")
print(data_dict)

README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11545it [00:00, 115438.28it/s]
Reading metadata...: 23375it [00:00, 117106.69it/s]
Reading metadata...: 35086it [00:00, 116352.49it/s]
Reading metadata...: 46722it [00:00, 116323.97it/s]
Reading metadata...: 58355it [00:00, 98757.75it/s] 
Reading metadata...: 68600it [00:00, 94304.43it/s]
Reading metadata...: 78278it [00:00, 93009.99it/s]
Reading metadata...: 88478it [00:00, 95604.73it/s]
Reading metadata...: 99397it [00:00, 99572.21it/s]
Reading metadata...: 109914it [00:01, 101218.26it/s]
Reading metadata...: 126570it [00:01, 101294.66it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6442it [00:00, 64390.74it/s]
Reading metadata...: 13448it [00:00, 61729.07it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 59333
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 12290
    })
})


Ushbu kod train va test bo‘limlaridan faqat dastlabki 10% ma'lumotlarni tanlab oladi. Bu hisoblash jarayonlarini tezlashtirish va resurslarni tejash uchun ishlatiladi. Bu jarayon har bir bo‘lim uchun select() funksiyasi yordamida amalga oshiriladi.

In [4]:
for split in ['train', 'test']:
    data_dict[split] = data_dict[split].select(range(int(len(data_dict[split]) * 0.1)))


In [5]:
print(data_dict["train"])

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 5933
})


Ushbu kod data_dict ma’lumot to‘plamidan kerakli bo‘lmagan ustunlarni o‘chiradi. Natijada faqat transkripsiya jarayoni uchun muhim ma’lumotlar saqlanadi. Bu ma’lumotlarni tozalash jarayoni modelni tayyorlashda xotira va hisoblash samaradorligini oshiradi.

In [6]:
data_dict = data_dict.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
data_dict

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 5933
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1229
    })
})

Ushbu kod data_dict ichidagi "audio" ustunini 16 kHz formatiga o‘tkazadi. Bu STT modellarida audio fayllarni to‘g‘ri qayta ishlash uchun muhim, chunki ularning ko‘pchiligi standart sifatida 16 kHz chastotani qabul qiladi. Bu jarayon modelni tayyorlash va tahlil qilish jarayonining muhim qismidir.

In [7]:
data_dict = data_dict.cast_column("audio", Audio(sampling_rate=16000))

Ushbu kod data_dict ma’lumot to‘plamidan berilgan indeksdagi audio faylni eshittiradi. U train yoki test bo‘limidan audio fayl yo‘lini oladi va IPython.display.Audio orqali audio namunani ijro etadi. Bu transkripsiya jarayonidan oldin ma’lumotlar sifatini tekshirishga yordam beradi.

In [8]:
# Audio namuna eshitish
def play_audio_from_dataset(data_dict, split='train', index=5):
    audio_path = data_dict[split][index]['audio']['path']
    print(f"Audio fayl eshittirilmoqda: {audio_path}")
    return ipd.Audio(audio_path)

audio_player = play_audio_from_dataset(data_dict, split='train', index=5)
audio_player


Audio fayl eshittirilmoqda: /root/.cache/huggingface/datasets/downloads/extracted/7bb9b71435ee139d70e3e87c5af0b6b04711807e25b9fe31d54ec7db3e0bc0a0/uz_train_0/common_voice_uz_28907257.mp3


Ushbu kod data_dict ma'lumot to‘plamidagi sentence ustunini tozalaydi. U maxsus belgilarni olib tashlaydi, jumlalarni kichik harflarga o‘tkazadi va bosh-oxiridagi bo‘sh joylarni o‘chiradi. map() funksiyasi yordamida tozalash barcha ma’lumotlarga qo‘llanadi, va show_one_example() funksiyasi yordamida asl va tozalangan matn taqqoslab ko‘rsatiladi. Bu transkripsiya sifatini yaxshilashga yordam beradi.

In [9]:
#-----------> DataText Celaning <-----------#

def show_one_example(data_dict, split='train', index=255):
    original_text = data_dict[split][index]['sentence']
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', original_text).strip().lower()

    print("Original Text --> ", original_text)
    print("Cleaned Text  --> ", cleaned_text)

# Bitta namunani ko‘rsatish
show_one_example(data_dict)

# datasetni cleaning yani tozalash bolimi
def clean_text(batch):
    batch['sentence'] = re.sub(r'[^a-zA-Z0-9\s]', '', batch['sentence']).strip().lower()
    return batch
data_dict = data_dict.map(clean_text)


Original Text -->  E, nega Shahribonu dadamdan qochmayapti, deb hayron bo‘larmishman.
Cleaned Text  -->  e nega shahribonu dadamdan qochmayapti deb hayron bolarmishman


Map:   0%|          | 0/5933 [00:00<?, ? examples/s]

Map:   0%|          | 0/1229 [00:00<?, ? examples/s]

In [10]:
print(data_dict["train"][152])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/7bb9b71435ee139d70e3e87c5af0b6b04711807e25b9fe31d54ec7db3e0bc0a0/uz_train_0/common_voice_uz_29253839.mp3', 'array': array([ 0.00000000e+00,  4.36557457e-11,  7.27595761e-12, ...,
        6.53811148e-05, -1.64598659e-05,  4.33773384e-05]), 'sampling_rate': 16000}, 'sentence': 'bogichingni boglab qoyishim mumkin'}


Ushbu kod audio va matnli ma’lumotlarni modelga kiritish uchun tayyorlaydi. Audio ma’lumotlar 16 kHz chastotaga resampl qilinib, feature_extractor() yordamida akustik xususiyatlar olinadi. Matnli ma’lumotlar esa tokenizer() orqali tokenlarga bo‘linadi va modelga mos formatga o‘tkaziladi. Bu jarayon STT modelini tayyorlash uchun muhim.

In [11]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


Ushbu kod data_dict ma’lumot to‘plamiga prepare_dataset() funksiyasini qo‘llab, har bir element uchun kerakli audio xususiyatlari va tokenlarini yaratadi. Barcha kerakmas ustunlar o‘chiriladi va faqat modelga kerakli xususiyatlar qoldiriladi. Bu STT modelini o‘rgatishga tayyorgarlik ko‘rish jarayonining muhim qismidir.

In [12]:
data_dict = data_dict.map(prepare_dataset, remove_columns=data_dict.column_names["train"], num_proc=1)

Map:   0%|          | 0/5933 [00:00<?, ? examples/s]

Map:   0%|          | 0/1229 [00:00<?, ? examples/s]

Ushbu kod DataCollatorSpeechSeq2SeqWithPadding klassini yaratadi, u modelga audio va matnli ma'lumotlarni birlashtiradi. Audio xususiyatlari va matnli tokenlar processor yordamida padding bilan standart uzunlikka keltiriladi va tensorlarga o‘tkaziladi. Bu jarayon STT modelini o‘rgatish uchun optimallashtirilgan ma’lumotlarni tayyorlashda muhimdir.

In [13]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Input va label xususiyatlarini ajratish
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        # Input xususiyatlarini padding bilan birlashtirish
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Label xususiyatlarini padding bilan birlashtirish va masklash
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # Agar barcha label pad token bilan boshlangan bo‘lsa, birinchi tokenni olib tashlash
        if torch.all(labels[:, 0] == self.processor.tokenizer.pad_token_id):
            labels = labels[:, 1:]

        # Batchga labelni qo‘shish
        batch["labels"] = labels
        return batch


Ushbu kod STT modelining WER (Word Error Rate) ko‘rsatkichini hisoblaydi. Modelning bashorati va haqiqiy matn token IDlari olinadi, pad tokenlar tiklanadi, va IDlar matn ko‘rinishiga o‘tkaziladi. WER hisoblanib, modelning ishlash natijasi foiz ko‘rinishida qaytariladi. Bu metrik modelning transkripsiya aniqligini baholash uchun ishlatiladi.

In [14]:
# Baholash metrikasini yuklash
import evaluate

# WER (Word Error Rate) baholash metrikasini yuklash
wer_errors = evaluate.load("wer")

# Baholash funksiyasini aniqlash
def compute_metrics(pred):
    # Bashorat qilingan va haqiqiy token IDlarini olish
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # -100 qiymatlarini pad_token_id bilan almashtirish
    label_ids = torch.where(label_ids == -100, processor.tokenizer.pad_token_id, label_ids)

    # Tokenlarni matnga aylantirish
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Word Error Rate (WER) hisoblash
    wer = wer_errors.compute(predictions=pred_str, references=label_str) * 100

    # Natijalarni qaytarish
    return {"wer": wer}


Ushbu kod STT modelini o‘rgatishda ma’lumotlarni tayyorlash uchun data_collator obyektini yaratadi. U audio va matnli tokenlarni padding bilan bir xil uzunlikka keltirib, modelga mos formatda saqlaydi. Bu modelning o‘qitish jarayonida barqaror ishlashini ta’minlaydi.

In [15]:

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Ushbu kod Whisper modelini FP16 formatda yuklash uchun ishlatiladi. FP16 format modelning xotira sarfini kamaytiradi va hisoblash tezligini oshiradi. device_map="auto" orqali model avtomatik ravishda GPU yoki CPU qurilmasiga joylashtiriladi. Bu modelni o‘rgatish va baholash jarayonini samarali qiladi.

In [16]:
from transformers import WhisperForConditionalGeneration

# Modelni FP16 formatda yuklash
model = WhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-small",
    torch_dtype=torch.float16,
    device_map="auto"  # GPU yoki CPU avtomatik tanlanadi
)


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

Ushbu kod Whisper modelini 8-bit kvantizatsiya va LoRA texnikasi bilan optimallashtiradi. gradient_checkpointing_enable() yordamida xotira sarfi kamaytiriladi. LoraConfig orqali LoRA sozlamalari o‘rnatilib, modelga faqat tanlangan qatlamlarga LoRA texnikasi qo‘llanadi. print_trainable_parameters() funktsiyasi modelda o‘rgatiladigan parametrlarning sonini ko‘rsatadi, bu hisoblash samaradorligini oshiradi va resurslarni tejaydi.

In [17]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(r = 32, lora_alpha=64, target_modules=["q_proj","v_proj"],lora_dropout=0.05, bias="none",)
model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.4429


Ushbu kod model o‘qitish va baholash uchun Seq2SeqTrainingArguments ni sozlaydi. Trening va baholash paytidagi partiya hajmlari, o‘qitish tezligi, epoch soni va loglash qadamlarining chastotasi kabi asosiy parametrlar aniqlangan. FP16 rejimi yoqilib, resurslardan samarali foydalanish ta'minlanadi. Bularning barchasi STT modelini yuqori samaradorlik bilan o‘rgatish uchun mo‘ljallangan.

In [18]:

# Trening argumentlarini sozlash
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-uz",
    max_steps=1000,
    per_device_train_batch_size=1,  # Batch hajmini kamaytiramiz
    gradient_accumulation_steps=4,  # Virtual batch hosil qilish
    learning_rate=1e-5,
    warmup_steps=50,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    per_device_eval_batch_size=32,  # Baholash uchun kichik batch
    generation_max_length=128,
    logging_steps=5,
    remove_unused_columns=False,
    label_names=["labels"],
    fp16=True,  # Mixed Precision rejimda trening
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Ushbu kod Seq2SeqTrainer yordamida STT modelini o‘qitish va baholashni amalga oshiradi. Model Whisper asosida yuklangan va LoRA bilan sozlangan. WER metrikasi yordamida natijalar baholanadi, train va test bo‘limlari esa ma’lumotlar to‘plami sifatida ishlatiladi. use_cache=False modeli keshlash xatolaridan himoya qiladi. Trening jarayonini boshlash uchun trainer.train() va baholash uchun trainer.evaluate() buyruqlaridan foydalaniladi.

In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=data_dict["train"],
    eval_dataset=data_dict["test"],
    data_collator=DataCollatorSpeechSeq2SeqWithPadding(processor=processor),
    compute_metrics=compute_metrics,  # To‘g‘ri baholash funksiyasini ulash
    tokenizer=processor.feature_extractor,
)

model.config.use_cache = False
import torch
torch.cuda.empty_cache()


<ipython-input-19-0fc3ce3f0cec>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


trainer.train() modelni o‘qitishni boshlaydi, har bir epoch oxirida modelni baholaydi va loglarni chop etadi. Modelning eng yaxshi versiyasi avtomatik saqlanadi va WER metrikasi asosida uning samaradorligi baholanadi. Bu STT modelini o‘qitish jarayonining asosiy bosqichidir.

In [21]:
# Modelni o‘qitish
trainer.train()



In [ ]:
# Modelni baholash
eval_results = trainer.evaluate()
print(f"WER: {eval_results['eval_wer']:.2f}%")


Ushbu kod modelni Hugging Face Hub-ga yuklash uchun ishlatiladi. Modelga berilgan nom va foydalanuvchi akkaunti birlashtirilib, push_to_hub() funksiyasi yordamida model serverga yuklanadi. Yuklash muvaffaqiyatli bo‘lsa, model identifikatori konsolga chiqariladi. Bu jarayon modelni ommaga taqdim qilish yoki keyinroq foydalanish uchun qulay saqlash imkonini beradi.

In [22]:
# model_hub_push = "STT_model_whisper_small"
# username = "AbdulxoliqMirzayev"
# peft_model_id = f"{username}/{model_hub_push}"
# model.push_to_hub(peft_model_id)

# print(f"Model successfully pushed to Hugging Face Hub: {peft_model_id}")


### **NER MODUL**

In [23]:
!pip install transformers datasets evaluate
!pip install seqeval
!pip install accelerate bitsandbytes
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
from datasets import load_dataset
from evaluate import load
import numpy as np
import pandas as pd
import torch
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=9f61dca28e63617ee1f6d8a37ebdf55c4107e8155b1d52653a67b097357a3a9e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Ushbu kod risqaliyevds/uzbek_ner datasetini yuklaydi.
Dataset trening va test bo‘limlariga ajratiladi (80% trening, 20% test).
Ajratilgan ma’lumotlar uzunligi chop etiladi.
Texnologiya: Hugging Face datasets kutubxonasi.

In [24]:
# Datasetni yuklash
dataset = load_dataset("risqaliyevds/uzbek_ner")

# Train datasetdan 80% trening uchun, 20% test uchun ajratish
train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=42)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

print(f"Train dataset uzunligi: {len(train_dataset)}")
print(f"Test dataset uzunligi: {len(test_dataset)}")


README.md:   0%|          | 0.00/3.05k [00:00<?, ?B/s]

uzbek_ner.json:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19609 [00:00<?, ? examples/s]

Train dataset uzunligi: 15687
Test dataset uzunligi: 3922


In [25]:
train_dataset.column_names
test_dataset.column_names

['ner', 'text']

Ushbu kod matnni tozalash uchun ishlatiladi: maxsus belgilar, URL manzillar, email manzillar va ortiqcha bo‘sh joylarni olib tashlaydi.
Matn tozalash uchun clean_text funksiyasi yaratilgan.
Namuna: Trening datasetidan matnning original va tozalangan ko‘rinishlari chop etiladi.

In [26]:

# Tozalash funksiyasi
import re

def clean_text(text):
    """
    Matnni tozalash funksiyasi:
    - Maxsus belgilarni olib tashlaydi
    - URL va email manzillarni olib tashlaydi
    - Ortib qolgan bo'sh joylarni tozalaydi
    """
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # URL larni olib tashlash
    text = re.sub(r"\S*@\S*\s?", '', text)  # Emailni olib tashlash
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Maxsus belgilarni olib tashlash
    text = re.sub(r'\s+', ' ', text).strip()  # Ortib qolgan bo'sh joylarni tozalash
    return text

# Namuna chop etish
def show_clean_vs_original(dataset, index=0):
    """
    Datasetdan original va tozalangan matnlarni chop etish
    """
    original_text = dataset[index]["text"]  # Original matn
    cleaned_text = clean_text(original_text)  # Tozalangan matn

    print("Original Text:")
    print(original_text)
    print("\nCleaned Text:")
    print(cleaned_text)

# Namuna ko‘rish
show_clean_vs_original(train_dataset, index=0)


Original Text:
Federal qidiruv byurosi direktori Jyeyms Komi San-Bernardinolik mergan Said Farukning smartfonini buzish haqida Apple’ga qo‘yilgan talab bundan ortiq global maqsadlarni nazarda tutmasligini bildirdi. “Biz kimningdir himoyasini buzmoqchi yoki qulf kalitini ommalashtirmoqchi emasmiz”, -dedi Komi. U milliy xavfsizlikka oid xayotiy muhim ma'lumotlar “kundalik ehtiyoj mahsulotlari bilan savdo qiluvchi” xususiy korporatsiyalarning qo‘lida saqlanishi noto‘g‘riligini ham bildirdi. Avvalroq, Kaliforniya sudi Apple kompaniyasiga San-Bernardinolik terroristning mobil telefoni qulfini yechishda yordam berishni buyurgandi. Tim Kuk Federal organlarning talabini bajarishdan bosh tortgan. Kukning fikricha, bu keyinchalik boshqa qurilmalarning xavfsizligiga ham daxl qilishi mumkin. Kaliforniya shtatining San-Bernardino shahrida imkoniyatlari cheklanganlarlar markazi hududida dekabrda sodir bo‘lgan otishmada 14 kishi halok bo‘lgan va 17 kishi yarador bo‘lgan edi.

Cleaned Text:
Federal qi

Ushbu kod train_dataset va test_dataset uchun matnni tozalash funksiyasini qo‘llaydi.
Har bir matn clean_text funksiyasi yordamida tozalanadi va qayta yoziladi.
map funksiyasi datasetning barcha elementlariga tozalash funksiyasini qo‘llaydi.

In [27]:
train_dataset = train_dataset.map(lambda x: {"text": clean_text(x["text"])})
test_dataset = test_dataset.map(lambda x: {"text": clean_text(x["text"])})


Map:   0%|          | 0/15687 [00:00<?, ? examples/s]

Map:   0%|          | 0/3922 [00:00<?, ? examples/s]

Ushbu kod datasetdagi barcha unikal NER teglarni aniqlash va indekslash uchun ishlatiladi.
extract_unique_tags funksiyasi datasetdagi barcha namunalardagi NER teglarni yig‘adi va ularni unikal ko‘rinishda saqlaydi.
Teglar label_list da saqlanadi va indekslash orqali tag_index lug‘ati hosil qilinadi (teglar indeks bilan bog‘lanadi).

In [28]:
# Datasetdagi barcha teglarni avtomatik aniqlash
def extract_unique_tags(dataset):
    unique_tags = set()
    for sample in dataset:
        unique_tags.update(sample["ner"])
    return sorted(unique_tags)

# Teglarni aniqlash
label_list = extract_unique_tags(train_dataset)
print("Teglar ro'yxati:", label_list)

# Teglarni indekslash
tag_index = {tag: i for i, tag in enumerate(label_list)}


Teglar ro'yxati: ['CARDINAL', 'DATE', 'EMAIL', 'EVENT', 'FAC', 'FACILITY', 'GPE', 'JCH-2022', 'JOURNAL', 'LANGUAGE', 'LAW', 'LOC', 'MISC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PER', 'PERCENT', 'PERIOD', 'PERSON', 'PHONE', 'PRODUCT', 'QUANTITY', 'RASUM', 'SOCIAL_MEDIA', 'TIME', 'WEBSITE', 'WORK_OF_ART']


Matnlarni tokenlash va NER teglarni tokenlarga moslashtirish orqali trening va test datasetlarini modelga mos shaklda tayyorlash.
Texnologiyalar:
Hugging Face Tokenizer (matnni tokenlash va o‘lchamini moslashtirish).
Datasetni qayta ishlash uchun map funksiyasi.
Teglarni indekslash (label_list, tag_index) va tokenlarga moslashtirish.

In [29]:
def extract_unique_tags(dataset):
    unique_tags = set()
    for sample in dataset:
        unique_tags.update(sample["ner"])
    return sorted(unique_tags)

# Teglarni aniqlash va indekslash
label_list = extract_unique_tags(train_dataset)
tag_index = {tag: i for i, tag in enumerate(label_list)}

# Har bir namunani tokenlash
def tokenize_sample(sample):
    tokenized_inputs = tokenizer(
        sample["text"].split(),
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )
    offset_mapping = tokenized_inputs.pop("offset_mapping")
    labels = []

    # Teglarni tokenlarga moslashtirish
    for offset, tag in zip(offset_mapping, sample["ner"]):
        if offset[0] == offset[1]:
            labels.append(-100)
        else:
            labels.append(tag_index[tag])

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Datasetni Tokenlash va Teglarni Moslashtirish
def preprocess_dataset(dataset):
    tokenized_dataset = dataset.map(tokenize_sample, batched=False)
    return tokenized_dataset


print("Trening datasetni tokenlash...")
tokenized_train_dataset = preprocess_dataset(train_dataset)

print("Test datasetni tokenlash...")
tokenized_test_dataset = preprocess_dataset(test_dataset)


Trening datasetni tokenlash...


Map:   0%|          | 0/15687 [00:00<?, ? examples/s]

Test datasetni tokenlash...


Map:   0%|          | 0/3922 [00:00<?, ? examples/s]

AutoModelForTokenClassification: NER uchun tayyor transformer modelini yuklash.
BERT (bert-base-multilingual-cased): Ko‘p tilli matnlarni ishlashga mos keladigan transformer modeli.
num_labels: Modelni NER teglar soniga moslashtirish.

In [30]:
# Modelni yuklash
model_name = "bert-base-multilingual-cased"
num_labels = len(label_list)

# NER modeli
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


evaluate kutubxonasi: seqeval metrikasi yordamida precision, recall, F1-score, va accuracy ni hisoblash.
compute_metrics funksiyasi:
Predictions va Labels: Model natijalarini va haqiqiy teglarni solishtiradi.
Filtering: -100 qiymatlarini chiqarib tashlaydi (ignor qilinadigan tokenlar uchun).
Natijalar: Metrika hisob-kitoblari natijalarni qaytaradi.

In [31]:
import evaluate

# Metrikani yuklash
metric = evaluate.load("seqeval")

# Metrikani hisoblash funksiyasi
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_model",
    evaluation_strategy="epoch",  # Har epoch oxirida baholash
    save_strategy="epoch",  # Har epoch oxirida saqlash
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,  # Faqat oxirgi 2 modelni saqlash
    load_best_model_at_end=True,  # Eng yaxshi modelni yuklash
    metric_for_best_model="f1"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Treningni boshlash
trainer.train()


<ipython-input-33-d766ad2b8aef>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


### S**TT AND NER PIPELINE MODUL**

aggregation_strategy="simple": NER natijalarida bir xil entitilarni birlashtirish.

STT modeli ovozli fayllarni transkriptsiya qiladi.
NER modeli matndagi entitilarni aniqlab, birlashtirilgan natijalarni qaytaradi.
STT va NER modellarni yuklash jarayonida avtomatik ravishda tokenizer bilan ishlaydi.

In [ ]:
from transformers import pipeline

# 1. STT va NER modellarni yuklash
def load_models(stt_model_path, ner_model_path):

    # STT modeli
    stt_pipeline = pipeline(
        "automatic-speech-recognition",
        model=stt_model_path,
        tokenizer=stt_model_path
    )

    # NER modeli
    ner_pipeline = pipeline(
        "ner",
        model=ner_model_path,
        tokenizer=ner_model_path,
        aggregation_strategy="simple"  # Entitilarni birlashtirish
    )

    return stt_pipeline, ner_pipeline


STT Pipeline (stt_pipeline): Ovozli fayldan matnni transkriptsiya qilish.
NER Pipeline (ner_pipeline): Transkriptsiya qilingan matndan entitilarni aniqlash.
Ishlash tartibi:

STT:
Ovozli fayl manzili (audio_path) orqali matn transkriptsiya qilinadi.
Natijada transkriptsiya qilingan matn olinadi.
NER:
Transkriptsiya qilingan matn NER modeli orqali ishlovdan o'tadi.
Matndagi entitilar (masalan, odam ismlari, joy nomlari va boshqalar) aniqlanadi.
Natijalar:

Transkriptsiya qilingan matn va aniqlangan entitilar lug‘at shaklida qaytariladi.

In [ ]:
#  Pipeline funksiyasini yaratish
def stt_to_ner_pipeline(audio_path, stt_pipeline, ner_pipeline):

    #  Ovozdan matnni transkriptsiya qilish
    print("Ovozdan matnni olish...")
    transcribed_text = stt_pipeline(audio_path)["text"]
    print(f"Transkriptsiya qilingan matn:\n{transcribed_text}")

    #  Matndan entitilarni aniqlash
    print("Matndan entitilarni aniqlash...")
    ner_results = ner_pipeline(transcribed_text)
    print(f"NER natijalari:\n{ner_results}")

    return {"transcribed_text": transcribed_text, "ner_results": ner_results}



In [ ]:
#  Model yo'llarini ko'rsatish

stt_model_path = "./whisper-small-uz"  # Fine-tuning qilingan STT model manzili
ner_model_path = "./ner_model"        # Fine-tuning qilingan NER model manzili



STT Pipeline (stt_pipeline): Yuklangan STT modeli orqali ovozli fayldan matn olish.
NER Pipeline (ner_pipeline): Matndagi entitilarni aniqlash.
Ishlash tartibi:

load_models: STT va NER modellarni yuklaydi.
audio_path: Sinov uchun ovozli fayl yo‘lini ko‘rsatadi.
stt_to_ner_pipeline:
Ovozli faylni transkriptsiya qilib matnni oladi (STT).
Transkriptsiya qilingan matndan entitilarni aniqlaydi (NER).
Natija:

Transkriptsiya qilingan matn va aniqlangan entitilar final_results sifatida qaytariladi.
Foydalanish: Ovozli fayl tahlilini oson va tez amalga oshirish uchun birlashtirilgan jarayon.

In [ ]:
#  Modellarni yuklash

stt_pipeline, ner_pipeline = load_models(stt_model_path, ner_model_path)


audio_path = "path_to_audio_file.wav"  # Sinov uchun ovozli fayl kiritsh kerak

#  Pipeline-ni ishlatish
final_results = stt_to_ner_pipeline(audio_path, stt_pipeline, ner_pipeline)



In [ ]:

# 7. Natijalarni chop etish
print("\n--- Yakuniy Natijalar ---")
print(f"Transkriptsiya qilingan matn:\n{final_results['transcribed_text']}")
print(f"NER natijalari:\n{final_results['ner_results']}")
